In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/TrainBienSo/charTrainset.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

mypath= "/tmp/charTrainset/"
characters=["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","G","H",
           "K","L","M","N","P","R","S","T","U","V","X","Y","Z"]
data=[]
for i,character in enumerate(characters):
    mypathAdded=mypath+character
    file_names=[f for f in listdir(mypathAdded) if isfile(join(mypathAdded, f))]
    for name in file_names:
        img=cv2.imread(mypathAdded+"/"+name,0)
        ret,img = cv2.threshold(img,50,255,cv2.THRESH_BINARY)
        data.append([img,i])
np.random.shuffle(data)
x_data=[]
y_data=[]
for [x, y] in data:
    x_data.append(x)
    y_data.append(y)
x_data=np.reshape(x_data,[-1,28,12,1])

In [0]:
import tensorflow as tf

def CNN_Model(input_shape):
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=(3,3),
                                     activation='relu',
                                     input_shape = input_shape
                                    ))
    model.add(tf.keras.layers.Conv2D(filters=64,
                                     kernel_size=(3,3),
                                     activation='relu'
                                    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=128,
                                    activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(units=31,
                                    activation='softmax'))
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                  optimizer=tf.keras.optimizers.Adam(), 
                  metrics=['accuracy'])
    return model

In [5]:
model=CNN_Model((28,12,1))
print(model.summary())

W0723 14:57:24.359290 140075587225472 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 10, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 8, 64)         18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 4, 64)         0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 4, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               393344    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [8]:
from time import time
import os
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
tensorboard= TensorBoard(log_dir="logs/{}".format(time()))

callbacks= [tensorboard]

history=model.fit(x_data, y_data, 
                  batch_size=10, 
                  epochs=100, 
                  validation_data=(x_data, y_data), 
                  shuffle=True,
                  callbacks=callbacks)
model.save("model.h5")

Train on 1076 samples, validate on 1076 samples
Epoch 1/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0363 - acc: 0.9870 - val_loss: 0.0020 - val_acc: 0.9991
Epoch 2/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0909 - acc: 0.9833 - val_loss: 6.0264e-06 - val_acc: 1.0000
Epoch 3/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0218 - acc: 0.9926 - val_loss: 9.5942e-08 - val_acc: 1.0000
Epoch 4/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0737 - acc: 0.9888 - val_loss: 0.0026 - val_acc: 0.9991
Epoch 5/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0539 - acc: 0.9916 - val_loss: 2.6821e-04 - val_acc: 1.0000
Epoch 6/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0910 - acc: 0.9842 - val_loss: 5.2321e-05 - val_acc: 1.0000
Epoch 7/100
1076/1076 [==============================] - 2s 2ms/sample - loss: 0.0966 - acc: 0.9777 - val_loss: 0.